# EPA1361 - Model-Based Decision Making

## Multi-model analysis

This exercise uses a simple version of the [Lotka-Volterra predator-prey equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) to show how the EMA Workbench can be used for a
multi-model analysis, in addition to typical parametric/structural uncertainties. This will let you test the connectors provided in the Workbench for Excel, NetLogo, and Vensim / PySD; we'll also use the models for the sensitivity analysis exercise in week 3.

* Using the three model files provided and the Python function below, define model objects for each implementation (Excel, NetLogo, Vensim/PySD, and Python), and test them using a single ensemble. Use 50 experiments sampled from the parameters below (so that each experiment will be executed for the 4 models, for a total of 200), and retrieve outputs for the _TIME_, _predators_, and _prey_ variables.
    * excel and vensim are only supported on windows
    * vensim requires the DSS version of Vensim
    * Netlogo supoprt depends on [jpype](http://jpype.readthedocs.io/en/latest/install.html) and [pynetlogo](https://pynetlogo.readthedocs.io/en/latest/). Also, if you don't have NetLogo installed, please get it from [NetLogo 6.1.1](https://ccl.northwestern.edu/netlogo/download.shtml) 
    * for pysd, see [its documentation](http://pysd.readthedocs.io/en/master/installation.html)
    * If possible try to work with all model versions, but even 2 or 3 (pure python and something else should be sufficient).
    

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |
|Final time	    |365	    |
|dt	    |0.25	    |

* Note that your EMA Workbench installation includes example scripts for the different connectors. The different model objects follow a similar syntax but will need to be slightly adjusted depending on the software (e.g. to specify the NetLogo run length or the sheet name in Excel). 

* These model objects can be used with a replication functionality (for instance to test the effect of stochastic uncertainty in a NetLogo model), which repeats a given experiment over multiple replications. You can use a single replication in this exercise as the models are not stochastic. By default, each outcome array will then have a shape of (# experiments, # replications, # time steps). Try adapting the outcome arrays so that they can be used with the _lines_ plotting function of the Workbench, and plot the results grouped by model.

* To check the graphical results, find the maximum absolute error of the time series you obtained for the _prey_ variable in the Excel, NetLogo, and Vensim/PySD models, relative to the Python function. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments,
                           ema_logging, SequentialEvaluator)

from ema_workbench.connectors.netlogo import NetLogoModel
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.connectors.pysd_connector import PysdModel

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis.plotting import lines, Density


def PredPrey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25, final_time=365, reps=1):

    #Initial values
    predators, prey, sim_time = [np.zeros((reps, int(final_time/dt)+1)) for _ in range(3)]
    
    for r in range(reps):
        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

        #Calculate the time series
        for t in range(0, sim_time.shape[1]-1):

            dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
            dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

            prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
            predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
            sim_time[r,t+1] = (t+1)*dt
    
    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

C:\Users\flori\Anaconda3\envs\epa1361\lib\site-packages\ema_workbench\connectors\__init__.py:17: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)


In [2]:
py_model = Model('PredPreyPy', function=PredPrey)

py_model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                      RealParameter('predator_efficiency', 0.0001, 0.004),
                      RealParameter('predator_loss_rate', 0.004, 0.08)]

py_model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

In [3]:
excel_model = ExcelModel('PredPreyExcel', wd=r'./model/', model_file='PredPrey.xlsx', default_sheet = 'Sheet1')

excel_model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                      RealParameter('predator_efficiency', 0.0001, 0.004),
                      RealParameter('predator_loss_rate', 0.004, 0.08)]

excel_model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

In [4]:
netlogo_model = NetLogoModel('PredPreyNetlogo', wd=r'./model/', model_file='PredPrey.nlogo')
netlogo_model.run_length = 1460
netlogo_model.replications = 1
netlogo_model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                      RealParameter('predator_efficiency', 0.0001, 0.004),
                      RealParameter('predator_loss_rate', 0.004, 0.08)]

netlogo_model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

In [5]:
pysd_model = PysdModel('PredPreyPysd', mdl_file=r'./model/PredPrey.mdl')

pysd_model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                      RealParameter('predator_efficiency', 0.0001, 0.004),
                      RealParameter('predator_loss_rate', 0.004, 0.08)]

pysd_model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

In [6]:
# we leave out netlogo model for now
ema_logging.log_to_stderr(ema_logging.INFO)
experiments, outcomes = perform_experiments([py_model, excel_model, pysd_model, netlogo_model],50)

[MainProcess/INFO] performing 50 scenarios * 1 policies * 4 model(s) = 200 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 60 cases completed
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 120 cases completed
[MainProcess/INFO] 140 cases completed
[MainProcess/INFO] 160 cases completed
[MainProcess/INFO] 180 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] experiments finished


In [7]:
for i in outcomes.keys():
    outcomes[i] = outcomes[i][:,0,:]

In [8]:
for outcome in outcomes.keys():
    if outcome == "Prey":
        lines(experiments, outcomes, outcomes_to_show = outcome, group_by = 'model', legend=True)

In [9]:
# merge the experiments with the outcomes so we can pairplot with seaborn
outcomesdf = pd.DataFrame(outcomes['prey'])
exp_outcomesdf = pd.merge(experiments,outcomesdf,left_index = True, right_index=True)

In [10]:
err = exp_outcomesdf.drop(columns=['predation_rate', 'predator_efficiency', 'predator_loss_rate', 'prey_birth_rate']).groupby('scenario').mad()

In [11]:
err.head()

,0,1,2,3,4,5,6,7,8,9,...,1451,1452,1453,1454,1455,1456,1457,1458,1459,1460
scenario,,,,,,,,,,,,,,,,,,,,,
0,0.0,5.329071e-15,0.000000e+00,5.329071e-15,1.776357e-15,1.776357e-15,1.776357e-15,1.776357e-15,1.776357e-15,1.776357e-15,...,1.243450e-14,1.243450e-14,1.154632e-14,1.154632e-14,1.154632e-14,1.243450e-14,1.243450e-14,1.243450e-14,1.243450e-14,1.243450e-14
1,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.329071e-15,5.329071e-15,1.776357e-15,1.776357e-15,5.329071e-15,...,3.552714e-15,3.552714e-15,7.105427e-15,3.552714e-15,3.552714e-15,3.552714e-15,5.329071e-15,1.776357e-15,1.776357e-15,0.000000e+00
2,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.776357e-15,5.329071e-15,5.329071e-15,5.329071e-15,...,3.552714e-15,3.552714e-15,4.440892e-15,4.440892e-15,4.440892e-15,4.440892e-15,4.440892e-15,4.440892e-15,4.440892e-15,4.440892e-15
3,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.776357e-15,5.329071e-15,5.329071e-15,...,3.730349e-14,3.730349e-14,3.730349e-14,3.819167e-14,3.819167e-14,3.819167e-14,3.819167e-14,3.819167e-14,3.819167e-14,3.819167e-14
4,0.0,1.776357e-15,1.776357e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.116363e-16,2.116363e-16,2.168404e-16,2.168404e-16,2.168404e-16,2.133710e-16,2.185752e-16,2.203099e-16,2.255141e-16,2.237793e-16


In [12]:
exp_outcomesdf

,predation_rate,predator_efficiency,predator_loss_rate,prey_birth_rate,scenario,policy,model,0,1,2,...,1451,1452,1453,1454,1455,1456,1457,1458,1459,1460
0,0.002289,0.002082,0.069115,0.024659,0,None,PredPreyPy,50.0,49.736070,49.468558,...,1.683573e+01,1.679069e+01,1.674705e+01,1.670477e+01,1.666384e+01,1.662424e+01,1.658596e+01,1.654898e+01,1.651328e+01,1.647885e+01
1,0.001175,0.001587,0.039070,0.031074,1,None,PredPreyPy,50.0,50.094780,50.186778,...,3.548005e+01,3.565843e+01,3.583729e+01,3.601662e+01,3.619640e+01,3.637662e+01,3.655727e+01,3.673835e+01,3.691983e+01,3.710170e+01
2,0.001036,0.001377,0.047015,0.017815,2,None,PredPreyPy,50.0,49.963610,49.925833,...,2.855131e+01,2.861696e+01,2.868288e+01,2.874907e+01,2.881553e+01,2.888226e+01,2.894925e+01,2.901651e+01,2.908404e+01,2.915183e+01
3,0.001425,0.003186,0.057611,0.023861,3,None,PredPreyPy,50.0,49.941990,49.875002,...,1.790134e+01,1.800274e+01,1.810472e+01,1.820727e+01,1.831041e+01,1.841413e+01,1.851844e+01,1.862333e+01,1.872882e+01,1.883491e+01
4,0.001897,0.002338,0.013624,0.018223,4,None,PredPreyPy,50.0,49.753497,49.496023,...,4.095138e-02,4.112179e-02,4.129296e-02,4.146491e-02,4.163762e-02,4.181111e-02,4.198537e-02,4.216042e-02,4.233625e-02,4.251287e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.002644,0.001450,0.045923,0.034264,45,None,PredPreyNetlogo,50.0,49.767250,49.531208,...,1.927255e+01,1.915456e+01,1.903856e+01,1.892453e+01,1.881244e+01,1.870227e+01,1.859400e+01,1.848762e+01,1.838310e+01,1.828041e+01
196,0.001232,0.003100,0.018405,0.031574,46,None,PredPreyNetlogo,50.0,50.086574,50.162760,...,2.519716e+00,2.539258e+00,2.558953e+00,2.578802e+00,2.598805e+00,2.618965e+00,2.639282e+00,2.659757e+00,2.680392e+00,2.701188e+00
197,0.000722,0.003379,0.008153,0.028275,47,None,PredPreyNetlogo,50.0,50.173040,50.339400,...,2.552642e-08,2.560558e-08,2.568518e-08,2.576525e-08,2.584577e-08,2.592675e-08,2.600819e-08,2.609009e-08,2.617247e-08,2.625531e-08
198,0.000583,0.001066,0.036059,0.025084,48,None,PredPreyNetlogo,50.0,50.167703,50.335337,...,1.648439e+01,1.652254e+01,1.656108e+01,1.660002e+01,1.663934e+01,1.667906e+01,1.671917e+01,1.675966e+01,1.680056e+01,1.684184e+01
